In [ ]:
import torch
import os
from pathlib import Path
#from logs.locallogger import LocalLogger3D
from logs.wandblogger import WandBLogger3D 
from training.trainer import MRTrainer
from datasets.signals import VolumeSignal, Procedural3DSignal
from datasets.utils import checker
from networks.mrnet import MRFactory
from datasets.pyramids import create_MR_structure
import yaml
from yaml.loader import SafeLoader
import matplotlib.pyplot as plt

In [ ]:
os.environ["WANDB_NOTEBOOK_NAME"] = "train3d.ipynb"
BASE_DIR = Path('.').absolute().parents[0]
VOXEL_PATH = BASE_DIR.joinpath('vox')
MODEL_PATH = BASE_DIR.joinpath('models')

In [ ]:
project_name = "dev-sandbox"
config_file = '../configs/config_3d_m_net.yml'
with open(config_file) as f:
    hyper = yaml.load(f, Loader=SafeLoader)
    if isinstance(hyper['batch_size'], str):
        hyper['batch_size'] = eval(hyper['batch_size'])
    print(hyper)
filepath = os.path.join(VOXEL_PATH, hyper['filename'])

In [ ]:
hyper['batch_size']

In [ ]:
dim = 256
# vol = torch.from_numpy(checker(dim, 32))
# base_signal = VolumeSignal(vol.view((1, dim, dim, dim)),
#                            hyper['domain'],
#                            batch_size=hyper['batch_size'])
# base_signal = VolumeSignal.init_fromfile(filepath,
#                                          hyper['domain'],
#                                          channels=hyper['channels'],
#                                          batch_size=hyper['batch_size'])

# train_dataloader = create_MR_structure(base_signal, hyper['max_stages'],
#                                         hyper['filter'], hyper['decimation'])
# test_dataloader = create_MR_structure(base_signal, hyper['max_stages'],
#                                         hyper['filter'], False)
proc = lambda x: torch.cos(x)
base_signal = Procedural3DSignal(
    proc,
    (dim, dim, dim),
    channels=hyper['channels'],
    domain=hyper['domain'],
    batch_size=hyper['batch_size']
)
train_dataloader = [base_signal]
test_dataloader = [base_signal]

In [ ]:
filename = os.path.basename(hyper['filename'])
wandblogger = WandBLogger3D(project_name,
                            f"{hyper['model']}{hyper['filter'][0].upper()}{filename[0:5]}",
                            hyper,
                            BASE_DIR)
mrmodel = MRFactory.from_dict(hyper)
print("Model: ", type(mrmodel))
mrtrainer = MRTrainer.init_from_dict(mrmodel, train_dataloader, test_dataloader, wandblogger, hyper)
mrtrainer.train(hyper['device'])